# 0. Setup.

In [8]:
%pip install -r ../../requirements_ai-prac.txt

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu126
   ---------------------------------------- 0.0/2.8 GB ? eta -:--:--
   ---------------------------------------- 0.0/2.8 GB 109.2 MB/s eta 0:00:26
    --------------------------------------- 0.0/2.8 GB 106.2 MB/s eta 0:00:26
    --------------------------------------- 0.1/2.8 GB 103.5 MB/s eta 0:00:27
   - -------------------------------------- 0.1/2.8 GB 105.9 MB/s eta 0:00:26
   - -------------------------------------- 0.1/2.8 GB 104.6 MB/s eta 0:00:26
   - -------------------------------------- 0.1/2.8 GB 105.2 MB/s eta 0:00:26
   -- ------------------------------------- 0.2/2.8 GB 104.6 MB/s eta 0:00:26
   -- ------------------------------------- 0.2/2.8 GB 104.8 MB/s eta 0:00:25
   -- ------------------------------------- 0.2/2.8 GB 105.2 MB/s eta 0:00:25
   --- ------------------------------------ 0.2/2.8 GB 105.5 MB/s eta 0:00:25
   --- ------------------------------------ 0.2/2.8 GB 105.9 MB/s e

  You can safely remove it manually.


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 1. Data.

In [3]:
df = pd.read_csv('data/train.csv')
df.head()

,row_id,body,rule,subreddit,positive_example_1,positive_example_2,negative_example_1,negative_example_2,rule_violation
0,0,Banks don't want you to know this! Click here ...,"No Advertising: Spam, referral links, unsolici...",Futurology,If you could tell your younger self something ...,hunt for lady for jack off in neighbourhood ht...,Watch Golden Globe Awards 2017 Live Online in ...,"DOUBLE CEE x BANDS EPPS - ""BIRDS""\n\nDOWNLOAD/...",0
1,1,SD Stream [ ENG Link 1] (http://www.sportsstre...,"No Advertising: Spam, referral links, unsolici...",soccerstreams,[I wanna kiss you all over! Stunning!](http://...,LOLGA.COM is One of the First Professional Onl...,#Rapper \n🚨Straight Outta Cross Keys SC 🚨YouTu...,[15 Amazing Hidden Features Of Google Search Y...,0
2,2,Lol. Try appealing the ban and say you won't d...,No legal advice: Do not offer or request legal...,pcmasterrace,Don't break up with him or call the cops. If ...,It'll be dismissed: https://en.wikipedia.org/w...,Where is there a site that still works where y...,Because this statement of his is true. It isn'...,1
3,3,she will come your home open her legs with an...,"No Advertising: Spam, referral links, unsolici...",sex,Selling Tyrande codes for 3€ to paypal. PM. \n...,tight pussy watch for your cock get her at thi...,NSFW(obviously) http://spankbang.com/iy3u/vide...,Good News ::Download WhatsApp 2.16.230 APK for...,1
4,4,code free tyrande --->>> [Imgur](http://i.imgu...,"No Advertising: Spam, referral links, unsolici...",hearthstone,wow!! amazing reminds me of the old days.Well...,seek for lady for sex in around http://p77.pl/...,must be watch movie https://sites.google.com/s...,We're streaming Pokemon Veitnamese Crystal RIG...,1


In [7]:
for r in df.iloc[0]:
    print(r)

0
Banks don't want you to know this! Click here to know more!
No Advertising: Spam, referral links, unsolicited advertising, and promotional content are not allowed.
Futurology
If you could tell your younger self something different about sex, what would that be?

i AM IN A CONTEST TO WIN FUNDING FOR MY SEX POSITIVE FILM: VOTE HERE:

http://sheknows.offerpop.com/campaign/813112/entry/v144417
hunt for lady for jack off in neighbourhood http://url.inmusi.com/gakq
Watch Golden Globe Awards 2017 Live Online in HD Coverage without ADS (VIP STREAMS)
=

HD STREAM QUALITY >>> [WATCH LINK1](http://forum.submitexpress.com/viewtopic.php?f=9&t=215858)
=

HD BROADCASTING QUALITY >>> [WATCH LINK1](http://forum.submitexpress.com/viewtopic.php?f=9&t=215858)
=

Mobile Compatibility: YES
=

NO ADS | NO ADS | ADS
=

DOUBLE CEE x BANDS EPPS - "BIRDS"

DOWNLOAD/STREAM:

http://music.theblacksmithed.com/download/birds/
0


# 2. Simple Trials.

## 2.1. `tf-idf` and `LogisticRegression`

In [12]:
# Cell 1: TF-IDF + Logistic Regression. Saves OOF and test predictions for blending.
# - Place data under ./data or the current directory; auto-detection included.

import os, gc, random, warnings, json
from pathlib import Path
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression as SkLR

# Reproducibility.
SEED = 42
def seed_all(s=SEED):
    random.seed(s); np.random.seed(s); os.environ["PYTHONHASHSEED"] = str(s)
seed_all()

# Locate data.
CAND_DATA_DIRS = [Path("data"), Path("."), Path("./jigsaw-agile-community-rules")]
DATA_DIR = next((p for p in CAND_DATA_DIRS if (p / "train.csv").exists() and (p / "test.csv").exists()), None)
assert DATA_DIR is not None, "Could not find train.csv and test.csv under ./data, . or ./jigsaw-agile-community-rules."

# Read data.
train = pd.read_csv(DATA_DIR / "train.csv").reset_index(drop=True)
test  = pd.read_csv(DATA_DIR / "test.csv").reset_index(drop=True)
sub   = pd.read_csv(DATA_DIR / "sample_submission.csv")

# Column picking.
TEXT_CANDS = ["body", "comment_text", "text", "comment"]
RULE_CANDS = ["rule", "rule_text", "rule_description"]
SUBR_CANDS = ["subreddit", "subreddit_name", "community"]
LABEL_CANDS = ["rule_violation", "target", "label"]

def pick_col(df, cands, required=True):
    for c in cands:
        if c in df.columns:
            return c
    if required:
        raise KeyError(f"Missing columns: need any of {cands}. Columns present: {list(df.columns)}.")
    return None

TEXT_COL  = pick_col(train, TEXT_CANDS)
RULE_COL  = pick_col(train, RULE_CANDS)
SUBR_COL  = pick_col(train, SUBR_CANDS)
LABEL_COL = pick_col(train, LABEL_CANDS)

# Rule-conditioned text.
def compose(df):
    return (
        df[RULE_COL].astype(str).fillna("") + " [SEP] " +
        df[SUBR_COL].astype(str).fillna("") + " [SEP] " +
        df[TEXT_COL].astype(str).fillna("")
    )

train["text_cond"] = compose(train)
test["text_cond"]  = compose(test)
y = train[LABEL_COL].astype(int).values

# 5-fold stratified CV.
N_FOLDS = 5
skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)
folds = list(skf.split(np.zeros(len(train)), y))

# TF-IDF + Logistic Regression.
tfidf = TfidfVectorizer(max_features=200_000, ngram_range=(1, 2), min_df=2)
X_all  = tfidf.fit_transform(train["text_cond"].tolist())
X_test = tfidf.transform(test["text_cond"].tolist())

oof_lr = np.zeros(len(train), dtype=float)
test_pred_lr = np.zeros(len(test), dtype=float)
auc_lr_folds = []

for fold, (tr, va) in enumerate(folds, 1):
    clf = SkLR(max_iter=1000, C=4.0, class_weight="balanced", solver="liblinear")
    clf.fit(X_all[tr], y[tr])
    oof_lr[va] = clf.predict_proba(X_all[va])[:, 1]
    auc = roc_auc_score(y[va], oof_lr[va])
    auc_lr_folds.append(auc)
    test_pred_lr += clf.predict_proba(X_test)[:, 1] / N_FOLDS

print(f"TF-IDF+LR fold AUCs: {np.round(auc_lr_folds, 4)} | mean={np.mean(auc_lr_folds):.4f}.")

# Save artifacts so Cell 2 can blend, even if run in a fresh kernel.
ARTIFACTS = Path("./artifacts"); ARTIFACTS.mkdir(exist_ok=True, parents=True)
np.save(ARTIFACTS / "oof_lr.npy", oof_lr)
np.save(ARTIFACTS / "test_pred_lr.npy", test_pred_lr)
meta = {
    "SEED": SEED, "N_FOLDS": N_FOLDS,
    "TEXT_COL": TEXT_COL, "RULE_COL": RULE_COL, "SUBR_COL": SUBR_COL, "LABEL_COL": LABEL_COL,
    "train_len": int(len(train)), "test_len": int(len(test))
}
with open(ARTIFACTS / "meta.json", "w") as f:
    json.dump(meta, f, indent=2)

print("Saved TF-IDF artifacts to ./artifacts/.")


TF-IDF+LR fold AUCs: [0.8389 0.7741 0.8309 0.8419 0.8003] | mean=0.8172.
Saved TF-IDF artifacts to ./artifacts/.


## 2.2. DeBERTa‑v3‑large.

In [4]:
# Cell 2: DeBERTa-v3-large fine-tune, then calibration + blending + submission.
# - Uses TF-IDF artifacts from memory or loads them from ./artifacts/.

import os, gc, random, warnings, json
from pathlib import Path
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression as LRCal

import torch
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    DataCollatorWithPadding, Trainer, TrainingArguments, EarlyStoppingCallback
)

# --------------------------------------------------------------------------------------
# Reproducibility and performance.
# --------------------------------------------------------------------------------------
SEED = 42
def seed_all(s=SEED):
    random.seed(s); np.random.seed(s)
    torch.manual_seed(s); torch.cuda.manual_seed_all(s)
    os.environ["PYTHONHASHSEED"] = str(s)
seed_all()

IS_WINDOWS = (os.name == "nt")

# Precision selector (mutually exclusive).
IS_CUDA = torch.cuda.is_available()
CC_MAJOR = torch.cuda.get_device_capability(0)[0] if IS_CUDA else 0
USE_BF16 = IS_CUDA and CC_MAJOR >= 8   # Ampere or newer.
USE_FP16 = IS_CUDA and not USE_BF16    # Pre-Ampere.

if IS_CUDA:
    # Safe speed boosts on NVIDIA GPUs.
    try:
        torch.set_float32_matmul_precision("high")
    except Exception:
        pass
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True

print(f"CUDA={IS_CUDA}, CC={CC_MAJOR if IS_CUDA else 'NA'}, bf16={USE_BF16}, fp16={USE_FP16}.")

# DataLoader workers: use 0 on Windows notebooks to avoid pickling issues.
DATALOADER_WORKERS = 0 if IS_WINDOWS else 4

# --------------------------------------------------------------------------------------
# Load data.
# --------------------------------------------------------------------------------------
def load_data():
    CAND_DATA_DIRS = [Path("data"), Path("."), Path("./jigsaw-agile-community-rules")]
    data_dir = next((p for p in CAND_DATA_DIRS if (p / "train.csv").exists() and (p / "test.csv").exists()), None)
    assert data_dir is not None, "Could not find train.csv and test.csv under ./data, . or ./jigsaw-agile-community-rules."
    train = pd.read_csv(data_dir / "train.csv").reset_index(drop=True)
    test  = pd.read_csv(data_dir / "test.csv").reset_index(drop=True)
    sub   = pd.read_csv(data_dir / "sample_submission.csv")
    return train, test, sub

if "train" not in globals() or "test" not in globals():
    train, test, sub = load_data()

# Column picking consistent with Cell 1.
TEXT_CANDS = ["body", "comment_text", "text", "comment"]
RULE_CANDS = ["rule", "rule_text", "rule_description"]
SUBR_CANDS = ["subreddit", "subreddit_name", "community"]
LABEL_CANDS = ["rule_violation", "target", "label"]

def pick_col(df, cands, required=True):
    for c in cands:
        if c in df.columns:
            return c
    if required:
        raise KeyError(f"Missing columns: need any of {cands}. Columns present: {list(df.columns)}.")
    return None

TEXT_COL  = pick_col(train, TEXT_CANDS)
RULE_COL  = pick_col(train, RULE_CANDS)
SUBR_COL  = pick_col(train, SUBR_CANDS)
LABEL_COL = pick_col(train, LABEL_CANDS)

# --------------------------------------------------------------------------------------
# Tokenizer and collator.
# --------------------------------------------------------------------------------------
MODEL_NAME = "microsoft/deberta-v3-large"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
SEP_TOKEN = tokenizer.sep_token or "[SEP]"
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")  # Ensure PyTorch tensors.

# Compose input text; keep simple, or swap SEP_TOKEN with actual special token.
def compose(df):
    return (
        df[RULE_COL].astype(str).fillna("") + f" {SEP_TOKEN} " +
        df[SUBR_COL].astype(str).fillna("") + f" {SEP_TOKEN} " +
        df[TEXT_COL].astype(str).fillna("")
    )

if "text_cond" not in train.columns:
    train["text_cond"] = compose(train)
if "text_cond" not in test.columns:
    test["text_cond"] = compose(test)

y = train[LABEL_COL].astype(int).values

# --------------------------------------------------------------------------------------
# CV splits.
# --------------------------------------------------------------------------------------
N_FOLDS = 3
skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)
folds = list(skf.split(np.zeros(len(train)), y))

# --------------------------------------------------------------------------------------
# Dataset.
# --------------------------------------------------------------------------------------
class DS(torch.utils.data.Dataset):
    def __init__(self, texts, labels=None):
        self.texts = texts; self.labels = labels
    def __len__(self):
        return len(self.texts)
    def __getitem__(self, i):
        enc = tokenizer(self.texts[i], truncation=True, max_length=320)
        if self.labels is not None:
            enc["labels"] = int(self.labels[i])
        return enc

# --------------------------------------------------------------------------------------
# Helpers.
# --------------------------------------------------------------------------------------
def logits_to_proba(logits):
    """Convert logits to P(class=1). Works for binary 1-logit or 2-logit outputs."""
    arr = np.asarray(logits)
    if arr.ndim == 1 or arr.shape[-1] == 1:
        z = arr.reshape(-1)
        return 1.0 / (1.0 + np.exp(-z))
    # Stable softmax for 2-class case.
    z = arr - arr.max(axis=1, keepdims=True)
    e = np.exp(z)
    return (e[:, 1] / e.sum(axis=1)).astype(np.float64)

def safe_logit(p, eps=1e-6):
    """Numerically stable logit."""
    p = np.clip(p, eps, 1 - eps)
    return np.log(p / (1 - p))

# --------------------------------------------------------------------------------------
# Train.
# --------------------------------------------------------------------------------------
oof_tr = np.zeros(len(train), dtype=float)
test_pred_tr = np.zeros(len(test), dtype=float)
auc_tr_folds = []

NUM_EPOCHS = 2
LR = 2e-5
TRAIN_BS = 24 if IS_CUDA else 8
EVAL_BS  = 48 if IS_CUDA else 16

for fold, (tr, va) in enumerate(folds, 1):
    ds_tr = DS(train["text_cond"].iloc[tr].tolist(), y[tr])
    ds_va = DS(train["text_cond"].iloc[va].tolist(), y[va])
    ds_ts = DS(test["text_cond"].tolist(), None)

    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME,
        num_labels=2,
        torch_dtype=(torch.bfloat16 if USE_BF16 else torch.float16 if USE_FP16 else torch.float32),
    )
    device = "cuda" if IS_CUDA else "cpu"
    model.to(device)
    model.gradient_checkpointing_disable()
    print(f"[Fold {fold}] Model device: {next(model.parameters()).device}, dtype: {next(model.parameters()).dtype}.")

    args = TrainingArguments(
        output_dir=f"./jigsaw_fold{fold}",
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="auc",
        greater_is_better=True,
        num_train_epochs=NUM_EPOCHS,
        learning_rate=LR,
        per_device_train_batch_size=TRAIN_BS,
        per_device_eval_batch_size=EVAL_BS,
        gradient_accumulation_steps=1,
        fp16=USE_FP16,
        bf16=USE_BF16,
        tf32=True,                      # Safe NVIDIA speed boost.
        no_cuda=not IS_CUDA,            # Force CUDA if available.
        dataloader_pin_memory=True,
        dataloader_num_workers=DATALOADER_WORKERS,
        group_by_length=True,           # Less padding, faster.
        gradient_checkpointing=False,   # Memory saver; Trainer will set use_cache=False.
        logging_steps=100,
        report_to=[],
        save_total_limit=1,
        seed=SEED,
        data_seed=SEED,
        auto_find_batch_size=False,     # Set True if you hit OOMs.
    )

    def compute_metrics(eval_pred):
        # Be robust to EvalPrediction or tuple.
        logits = getattr(eval_pred, "predictions", None)
        labels = getattr(eval_pred, "label_ids", None)
        if logits is None or labels is None:
            logits, labels = eval_pred
        probs = logits_to_proba(logits)
        return {"auc": roc_auc_score(labels, probs)}

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=ds_tr,
        eval_dataset=ds_va,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=1, early_stopping_threshold=0.0001)],
    )

    trainer.train()
    val_logits = trainer.predict(ds_va).predictions
    oof_tr[va] = logits_to_proba(val_logits)
    auc = roc_auc_score(y[va], oof_tr[va])
    auc_tr_folds.append(auc)

    ts_logits = trainer.predict(ds_ts).predictions
    test_pred_tr += logits_to_proba(ts_logits) / N_FOLDS

    del trainer, model, ds_tr, ds_va, ds_ts
    gc.collect()
    if IS_CUDA:
        torch.cuda.empty_cache()

print(f"DeBERTa fold AUCs: {np.round(auc_tr_folds, 4)} | mean={np.mean(auc_tr_folds):.4f}.")

# --------------------------------------------------------------------------------------
# Calibration and blending.
# --------------------------------------------------------------------------------------
ARTIFACTS = Path("./artifacts")
def load_or_none(path):
    return np.load(path) if path.exists() else None

if "oof_lr" not in globals() or "test_pred_lr" not in globals():
    oof_lr = load_or_none(ARTIFACTS / "oof_lr.npy")
    test_pred_lr = load_or_none(ARTIFACTS / "test_pred_lr.npy")

use_lr_view = (
    (oof_lr is not None) and (test_pred_lr is not None)
    and (len(oof_lr) == len(train)) and (len(test_pred_lr) == len(test))
)

if use_lr_view:
    logit_lr = safe_logit(oof_lr)
    logit_tr = safe_logit(oof_tr)
    X_cal = np.vstack([logit_lr, logit_tr]).T
    cal = LRCal(max_iter=200)
    cal.fit(X_cal, y)
    oof_blend = cal.predict_proba(X_cal)[:, 1]
    print(f"Blended OOF AUC: {roc_auc_score(y, oof_blend):.4f}.")
    test_logit_lr = safe_logit(test_pred_lr)
    test_logit_tr = safe_logit(test_pred_tr)
    X_test_blend = np.vstack([test_logit_lr, test_logit_tr]).T
    test_pred = cal.predict_proba(X_test_blend)[:, 1]
else:
    # Fallback to calibrated transformer only.
    X_cal = safe_logit(oof_tr).reshape(-1, 1)
    cal = LRCal(max_iter=200).fit(X_cal, y)
    oof_cal = cal.predict_proba(X_cal)[:, 1]
    print(f"Transformer-only calibrated OOF AUC: {roc_auc_score(y, oof_cal):.4f}.")
    test_pred = cal.predict_proba(safe_logit(test_pred_tr).reshape(-1, 1))[:, 1]

# --------------------------------------------------------------------------------------
# Save submission.
# --------------------------------------------------------------------------------------
assert "row_id" in test.columns, "Expected 'row_id' in test.csv."
out = pd.DataFrame({"row_id": test["row_id"], "rule_violation": test_pred})
assert len(out) == len(test), "Submission length must equal test length."
assert np.isfinite(out["rule_violation"]).all(), "Predictions must be finite."
out.to_csv("submission.csv", index=False)
print("Wrote submission.csv with shape:", out.shape)


CUDA=True, CC=12, bf16=True, fp16=False.


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 2, 'bos_token_id': 1}.


[Fold 1] Model device: cuda:0, dtype: torch.bfloat16.


Epoch,Training Loss,Validation Loss,Auc
1,No log,0.692571,0.668192
2,0.697700,0.691114,0.666505


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 2, 'bos_token_id': 1}.


...

Epoch,Training Loss,Validation Loss,Auc
1,No log,0.672414,0.638349
2,0.684200,0.670531,0.641480


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 2, 'bos_token_id': 1}.


[Fold 5] Model device: cuda:0, dtype: torch.bfloat16.


Epoch,Training Loss,Validation Loss,Auc
1,No log,0.681870,0.598795
2,0.686200,0.680616,0.605064


DeBERTa fold AUCs: [0.5014 0.4953 0.516  0.5025 0.5156] | mean=0.5062.
Blended OOF AUC: 0.8174.
Wrote submission.csv with shape: (10, 2)


In [5]:
SAVE_DIR = "deberta_v3_large_finetuned"
trainer.save_model(SAVE_DIR)         # Saves model weights + config.
tokenizer.save_pretrained(SAVE_DIR)  # Saves tokenizer files.


NameError: name 'trainer' is not defined